<h1 style="text-align: center;" markdown="1">TP2: Random Forest tuneo de parametros</h1>
![](../data/icon_properati-data.png)

In [1]:
from sklearn import svm
import numpy as np 
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV

import pandas as pd

# modules
import knn as knnlibrary

/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
properties =knnlibrary.get_dataset()
properties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374568 entries, 0 to 34525
Data columns (total 27 columns):
id                            374568 non-null object
created_on                    374568 non-null object
operation                     374568 non-null object
property_type                 374568 non-null object
place_name                    374483 non-null object
place_with_parent_names       374568 non-null object
country_name                  374568 non-null object
state_name                    374568 non-null object
geonames_id                   309976 non-null float64
lat-lon                       311335 non-null object
lat                           311335 non-null float64
lon                           311335 non-null float64
price                         333523 non-null float64
currency                      324061 non-null object
price_aprox_local_currency    333523 non-null float64
price_aprox_usd               333523 non-null float64
surface_total_in_m2           260093

In [3]:
# filtro por CABA y GBA
properties_caba = properties[(properties['place_with_parent_names'].str.contains('Capital Federal') \
                             | properties['place_with_parent_names'].str.contains('Bs.As. G.B.A.'))]

properties_caba = properties_caba[properties_caba['price'].notnull() & properties_caba['price'] > 0 & properties_caba['place_name'].notnull()]

# eliminamos propiedades con mas de 54 pisos
properties_caba = properties_caba[properties_caba['floor'] <= 60]
# eliminamos propiedades con mas de 9 pisos - ver analisis
properties_caba = properties_caba[properties_caba['rooms'] <=  10]
# eliminamos propiedades con mas de 2000 m2 de superficie cubierta - ver analisis
properties_caba = properties_caba[(properties_caba['surface_covered_in_m2'] <= 4000) & (properties_caba['surface_covered_in_m2'] >= 0)]
# eliminamos propiedades con mas de 2000 m2 de superficie cubierta - ver analisis
properties_caba = properties_caba[(properties_caba['surface_total_in_m2']<= 6000)  & (properties_caba['surface_total_in_m2'] >= 0)]
properties_caba.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9204 entries, 16 to 34504
Data columns (total 27 columns):
id                            9204 non-null object
created_on                    9204 non-null object
operation                     9204 non-null object
property_type                 9204 non-null object
place_name                    9204 non-null object
place_with_parent_names       9204 non-null object
country_name                  9204 non-null object
state_name                    9204 non-null object
geonames_id                   8045 non-null float64
lat-lon                       8500 non-null object
lat                           8500 non-null float64
lon                           8500 non-null float64
price                         9204 non-null float64
currency                      9204 non-null object
price_aprox_local_currency    9204 non-null float64
price_aprox_usd               9204 non-null float64
surface_total_in_m2           9204 non-null float64
surface_covered_i

In [4]:
# filtro columnas segun lo que hay en el dataset
attributes = ['created_on','property_type','operation','place_name','place_with_parent_names',\
              'country_name','state_name','lat','lon','surface_total_in_m2','surface_covered_in_m2',\
              'floor','rooms', 'price']
properties_caba[attributes].info()
properties_caba_with_price_attributes = properties_caba[attributes]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9204 entries, 16 to 34504
Data columns (total 14 columns):
created_on                 9204 non-null object
property_type              9204 non-null object
operation                  9204 non-null object
place_name                 9204 non-null object
place_with_parent_names    9204 non-null object
country_name               9204 non-null object
state_name                 9204 non-null object
lat                        8500 non-null float64
lon                        8500 non-null float64
surface_total_in_m2        9204 non-null float64
surface_covered_in_m2      9204 non-null float64
floor                      9204 non-null float64
rooms                      9204 non-null float64
price                      9204 non-null float64
dtypes: float64(7), object(7)
memory usage: 1.1+ MB


In [5]:
from sklearn.preprocessing import LabelEncoder

properties_caba_with_price_attributes.fillna('NaN', inplace=True)
# atributos categoricos
encoder = LabelEncoder()

properties_caba_with_price_attributes['property_type'] = encoder.fit_transform(properties_caba_with_price_attributes[['property_type']])
properties_caba_with_price_attributes['operation'] = encoder.fit_transform(properties_caba_with_price_attributes[['operation']])
properties_caba_with_price_attributes['place_name'] = encoder.fit_transform(properties_caba_with_price_attributes[['place_name']])
properties_caba_with_price_attributes['place_with_parent_names'] = encoder.fit_transform(properties_caba_with_price_attributes[['place_with_parent_names']])
properties_caba_with_price_attributes['country_name'] = encoder.fit_transform(properties_caba_with_price_attributes[['country_name']])
properties_caba_with_price_attributes['state_name'] = encoder.fit_transform(properties_caba_with_price_attributes[['state_name']])
properties_caba_with_price_attributes['created_on'] = encoder.fit_transform(properties_caba_with_price_attributes[['created_on']])
# completamos valores nan
from sklearn.preprocessing import Imputer
imputer_mean = Imputer(missing_values='NaN', strategy='mean', axis=0)
properties_caba_with_price_attributes["lat"] = imputer_mean.fit_transform(properties_caba_with_price_attributes[["lat"]])
properties_caba_with_price_attributes["lon"] = imputer_mean.fit_transform(properties_caba_with_price_attributes[["lon"]])

properties_caba_with_price_attributes.head()

/opt/conda/lib/python3.5/site-packages/pandas/core/frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A

,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,price
16,133,1,0,129,69,0,1,-34.680247,-58.553347,59.0,55.0,1.0,2.0,93000.0
17,133,1,0,129,69,0,1,-34.680247,-58.553347,59.0,55.0,2.0,2.0,93000.0
18,133,1,0,129,69,0,1,-34.680247,-58.553347,59.0,55.0,3.0,2.0,93000.0
19,133,1,0,129,69,0,1,-34.680247,-58.553347,59.0,55.0,4.0,2.0,93000.0
20,133,1,0,129,69,0,1,-34.680247,-58.553347,55.0,55.0,1.0,2.0,93000.0


In [6]:
# separamos el set de train
from sklearn.model_selection import train_test_split
import datetime

now = datetime.datetime.now()

# separamos el train de traing para validarlo luego usando un 30% de los datos
X, y = properties_caba_with_price_attributes.iloc[:, properties_caba_with_price_attributes.columns != 'price'].values, properties_caba_with_price_attributes.iloc[:, properties_caba_with_price_attributes.columns == 'price'].values
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.2, random_state=now.microsecond)
    
print(y_train)
    
len(X_test), len(X_train)

[[ 104000.]
 [ 214719.]
 [ 165000.]
 ..., 
 [ 186000.]
 [  68000.]
 [ 150000.]]


(1841, 7363)

In [7]:
# preprocesamiento
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

scaler = preprocessing.StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

pipeline = make_pipeline(preprocessing.StandardScaler(), RandomForestRegressor(n_estimators=100))

In [10]:
# hiperparametros a tunear
print(pipeline.get_params())

hyperparameters = { 'randomforestregressor__max_depth': [10,30,50,70,90,100],
                    'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                    'randomforestregressor__min_samples_leaf': [5,10,15,20],
                    'randomforestregressor__min_samples_split': [5,10,15,20]}

{'randomforestregressor__min_impurity_decrease': 0.0, 'randomforestregressor__bootstrap': True, 'randomforestregressor__max_leaf_nodes': None, 'randomforestregressor__verbose': 0, 'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'randomforestregressor__n_estimators': 100, 'randomforestregressor__warm_start': False, 'randomforestregressor__criterion': 'mse', 'standardscaler__with_mean': True, 'steps': [('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))], 'randomforestregressor__min_weight_fraction_leaf': 0.0, 'randomforestregresso

In [ ]:
# crossvalidation
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
 
# Fit and tune model
clf.fit(X_train, y_train)

/opt/conda/lib/python3.5/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/opt/conda/lib/python3.5/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/opt/conda/lib/python3.5/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/opt/conda/lib/python3.5/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for 

In [ ]:
# cuales son los mejores hiperparametros
print(clf.best_params_)

#Confirm model will be retrained
print(clf.refit)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

y_pred = clf.predict(X_test)

print(r2_score(y_test, y_pred))
 
print(mean_squared_error(y_test, y_pred))

In [ ]:
# Creando modelo con hiper-parametros encontrados
model = RandomForestRegressor(n_estimators=1500, max_features='sqrt', max_depth=,min_samples_leaf=,min_samples_split= )
model.fit(Xtrn, Ytrn)

r2_score(Ytest, model.predict(Xtest))

# Prediction

In [4]:
# leemos set de test
test_df = pd.read_csv('../data/test/properati_dataset_testing_noprice.csv', low_memory=False)
test_df.head()

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.0,NaN,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.0,48.0,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,51.0,51.0,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...


In [5]:
# transformamos atributos categoricos
test_df['place_name'] = encoder.fit_transform(test_df[['place_name']])
test_df['state_name'] = encoder.fit_transform(test_df[['state_name']])
test_df['place_with_parent_names'] = encoder.fit_transform(test_df[['place_with_parent_names']])
test_df['property_type'] = encoder.fit_transform(test_df[['property_type']])

# tranformamos fechas
X_test_df = knnlibrary.transform_date(test_df)
X_test_df = X_test_df[['floor', 'lat', 'lon', 'place_name', 'place_with_parent_names',
       'property_type', 'rooms', 'state_name', 'surface_covered_in_m2',
       'surface_total_in_m2', 'created_on_year', 'created_on_month',
       'created_on_day']]

# completamos valores nan
from sklearn.preprocessing import Imputer
imputer_mean = Imputer(missing_values='NaN', strategy='mean', axis=0)
X_test_df['floor'] = X_test_df[['floor']].fillna(1)
X_test_df['rooms'] = X_test_df[['rooms']].fillna(1)

X_test_df["surface_total_in_m2"] = imputer_mean.fit_transform(X_test_df[["surface_total_in_m2"]])
X_test_df["surface_covered_in_m2"] = imputer_mean.fit_transform(X_test_df[["surface_covered_in_m2"]])
X_test_df["lat"] = imputer_mean.fit_transform(X_test_df[["lat"]])
X_test_df["lon"] = imputer_mean.fit_transform(X_test_df[["lon"]])

#X_test_std_df = stdsc.transform(X_test_df)

X_test_df.head()

/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:20:

,floor,lat,lon,place_name,place_with_parent_names,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,created_on_year,created_on_month,created_on_day
0,1.0,-34.610988,-58.363464,152,219,1,1.0,4,359.471588,0.0,2017,8,24
1,1.0,-34.629923,-58.465820,29,180,1,1.0,3,359.471588,0.0,2017,8,25
2,1.0,-34.589363,-58.412880,136,210,1,1.0,4,48.000000,53.0,2017,8,1
3,1.0,-34.629923,-58.465820,40,193,1,1.0,4,39.000000,39.0,2017,8,1
4,1.0,-34.629923,-58.465820,40,193,1,1.0,4,51.000000,51.0,2017,8,1


In [6]:
# prediccion
X_test_df['prediction'] = model.predict(X_test_df)
X_test_df.head(10)

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,floor,lat,lon,place_name,place_with_parent_names,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,created_on_year,created_on_month,created_on_day,prediction
0,1.0,-34.610988,-58.363464,152,219,1,1.0,4,359.471588,0.0,2017,8,24,228160.093267
1,1.0,-34.629923,-58.465820,29,180,1,1.0,3,359.471588,0.0,2017,8,25,224175.770067
2,1.0,-34.589363,-58.412880,136,210,1,1.0,4,48.000000,53.0,2017,8,1,320276.131800
3,1.0,-34.629923,-58.465820,40,193,1,1.0,4,39.000000,39.0,2017,8,1,320276.131800
4,1.0,-34.629923,-58.465820,40,193,1,1.0,4,51.000000,51.0,2017,8,1,320276.131800
5,1.0,-34.629923,-58.465820,40,193,1,1.0,4,39.000000,39.0,2017,8,1,320276.131800
6,1.0,-34.629923,-58.465820,40,193,1,1.0,4,39.000000,39.0,2017,8,1,320276.131800
7,1.0,-34.629923,-58.465820,40,193,1,1.0,4,39.000000,39.0,2017,8,1,320276.131800
8,1.0,-34.578721,-58.457095,47,195,1,1.0,4,67.000000,67.0,2017,8,1,320276.131800
9,1.0,-34.629923,-58.465820,7,184,1,1.0,4,47.000000,47.0,2017,8,1,320276.131800


In [7]:
output = pd.DataFrame( data={"id":test_df["id"], "price_usd":X_test_df['prediction']} )

In [8]:
# guardamos
output.to_csv( "../data/result/result_randomforest_"+str(now)+".csv", index=False, quoting=3 )